In [9]:
import matplotlib
from bert_score import score
import torch


In [ ]:

import pandas as pd
from bert_score import score
import torch


In [75]:
import pandas as pd
from bert_score import score
import torch

def read_excel_first_sheet(file_path: str) -> pd.DataFrame:
    """
    Reads the first sheet of an Excel file into a DataFrame.
    """
    df = pd.read_excel(file_path, sheet_name=0)  # 'sheet_name=0' reads the first sheet
    return df

def add_bert_f1_to_dataframe(df: pd.DataFrame, col1: str, col2: str, score_col: str) -> pd.DataFrame:
    """
    Adds a new column with BERT F1 scores to the DataFrame.
    """
    # Extract the ground truth and predicted columns from the DataFrame
    gt_list = df[col1].tolist()
    print (gt_list)
    pt_list = df[col2].tolist()
    
    print (pt_list)
    # Calculate BERT F1 scores
    _, _, f1 = score(gt_list, pt_list, lang="en", model_type="bert-base-uncased", batch_size=64)
    
    # Convert the torch.Tensor to a list and add it as a new column
    df[score_col] = f1.tolist()
    
    return df

import pandas as pd
from itertools import combinations
from bert_score import score
import torch

def add_pairwise_bert_f1_to_dataframe(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    Adds new columns with BERT F1 scores for each pairwise combination of the specified columns.
    The new columns are named '<col1>:<col2>'.
    
    :param df: The input DataFrame.
    :param columns: A list of four column names to compare pairwise.
    :return: The DataFrame with new BERT F1 score columns added.
    """
    # Generate all pairwise combinations of the columns
    pairs = list(combinations(columns, 2))
    print (pairs)
    # For each pair of columns, calculate BERT F1 scores and add as new columns
    for col1, col2 in pairs:
        
        gt_list = df[col1].tolist()
        pt_list = df[col2].tolist()
   
        # Calculate BERT F1 scores for the pair
        _, _, f1 = score([str(g) for g in gt_list], [str(p) for p in pt_list], lang="en", model_type="bert-base-uncased", batch_size=64)
        
        # Add the F1 scores as a new column with the name "<col1>:<col2>"
        score_col = f"{col1}:{col2}"
        df[score_col] = f1.tolist()
    
    return df

def update_excel_with_pairwise_bert_f1(input_excel: str, columns: list, output_excel: str = None):
    """
    Reads the first sheet of an Excel file, adds pairwise BERT F1 scores for four columns, 
    and saves the updated DataFrame back to Excel.
    
    :param input_excel: Path to the input Excel file.
    :param columns: A list of four column names to compare pairwise.
    :param output_excel: Path to the output Excel file. If None, overwrites the input file.
    """
    # Step 1: Read the first sheet of the Excel file
    df = read_excel_first_sheet(input_excel)
    
    # Step 2: Add pairwise BERT F1 scores to the DataFrame
    df = add_pairwise_bert_f1_to_dataframe(df, columns)
    
    # Step 3: Save the updated DataFrame back to Excel
    if output_excel is None:
        output_excel = input_excel
    
    with pd.ExcelWriter(output_excel, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')  # Save back to the first sheet
    
    print(f"Updated Excel file saved to {output_excel}")

# Example usage:
# update_excel_with_pairwise_bert_f1('input.xlsx', ['col1', 'col2', 'col3', 'col4'])


In [76]:
update_excel_with_pairwise_bert_f1('consistency.xlsx', ['Response_1', 'Response_2', 'Response_3', 'Response_4'])

[('Response_1', 'Response_2'), ('Response_1', 'Response_3'), ('Response_1', 'Response_4'), ('Response_2', 'Response_3'), ('Response_2', 'Response_4'), ('Response_3', 'Response_4')]
Updated Excel file saved to consistency.xlsx


In [44]:
# update_excel_with_pairwise_bert_f1('nvinfo_consistency_before.xlsx', ['Response1', 'Response2', 'Response3', 'Response4'])

In [51]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import pandas as pd
from collections import Counter
SEPERATOR = ":"


def find_max_count(strings):
    # Count occurrences of each string in the list
    string_counts = Counter(strings)
    
    # Find the maximum count
    max_count = max(string_counts.values())
    
    # Find all strings that have the maximum count
    most_common_strings = [string for string, count in string_counts.items() if count == max_count]
    
    return most_common_strings, max_count

def label_outliers(df: pd.DataFrame, columns: list, threshold: float) -> pd.DataFrame:
    """
    Adds a new column 'Outlier' to the DataFrame. Marks columns as outliers if any pairwise comparison
    score is below the threshold. For multiple columns, combines the names of columns identified as outliers.
    
    :param df: The input DataFrame.
    :param columns: A list of column names used in pairwise comparisons.
    :param threshold: The threshold value for outlier detection.
    :return: The DataFrame with the new 'Outlier' column.
    """
    # Initialize a dictionary to hold outlier status for each row
    outlier_dict = {col: [] for col in columns}
    
    # Determine which columns should be considered based on the threshold
    for index, row in df.iterrows():
        outlier_columns = set()
        outlier_column_info = []
        for col in columns:
            if row[col] < threshold:
                outlier_columns.add(col)
                for col2 in col.split(SEPERATOR):
                    outlier_column_info.append(col2)
        # # Update outlier_dict for this row
        # for col in outlier_columns:
        #     outlier_dict[col].append(True)
        # if not outlier_columns:
        #     outlier_dict[col].append(False)
        # 

        df.at[index, 'Outlier Response'] = ', '.join([column_name for column_name in find_max_count(outlier_column_info)[0]])
        # df['Outlier Response'] = ', '.join(outlier_column_info)
    # Create the 'Outlier' column as a combination of the column names that are outliers
    df['Outlier'] = df[columns].apply(lambda row: any(value < threshold for value in row), axis=1)

    return df


def apply_color_to_outliers(output_excel: str, df: pd.DataFrame, columns: list, threshold: float):
    """
    Applies light red color formatting to cells below the threshold in the outlier columns.

    :param output_excel: The Excel file to apply formatting to.
    :param df: The DataFrame containing the data.
    :param columns: The list of column names used for outlier detection.
    :param threshold: The threshold value for outlier detection.
    """
    # Load the Excel file
    wb = load_workbook(output_excel)
    ws = wb.active

    # Define a light red fill for cells below the threshold
    red_fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")

    # Apply the color to the relevant cells
    for col in columns:
        for row in range(2, len(df) + 2):  # Excel rows start at 1, skip the header
            cell_value = ws[f'{col}{row}'].value
            if isinstance(cell_value, (int, float)) and cell_value < threshold:
                ws[f'{col}{row}'].fill = red_fill

    # Save the workbook with the new formatting
    wb.save(output_excel)
    
def save_outliers_to_excel(input_excel: str, columns: list, threshold: float, output_excel: str = None):
    """
    Reads the first sheet of an Excel file, adds pairwise BERT F1 scores, labels outliers,
    creates a second sheet with only outliers, and saves the updated DataFrame back to Excel.
    
    :param input_excel: Path to the input Excel file.
    :param columns: A list of four column names to compare pairwise.
    :param threshold: The threshold value for outlier detection.
    :param output_excel: Path to the output Excel file. If None, overwrites the input file.
    """
    # Step 1: Read the first sheet of the Excel file
    
    df = read_excel_first_sheet(input_excel)
    
    # Step 2: Add pairwise BERT F1 scores to the DataFrame
    df = add_pairwise_bert_f1_to_dataframe(df, columns)
    
    # Generate BERT F1 score columns for outlier detection
    score_columns = [f"{col1}{SEPERATOR}{col2}" for col1, col2 in combinations(columns, 2)]
    
    # Step 3: Label outliers
    if threshold:
        df = label_outliers(df, score_columns, threshold)
    
    # Create a DataFrame with only outliers
    # outliers_df = df[df['Outlier'] != 'FALSE']
    
    # Step 4: Save the updated DataFrame and outliers to Excel
    if output_excel is None:
        output_excel = input_excel
    
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        # Save the main DataFrame with 'Outlier' column
        df.to_excel(writer, index=False, sheet_name='Sheet1')
        
        # Save the outliers DataFrame to a new sheet
        # outliers_df.to_excel(writer, index=False, sheet_name='Outliers')
    apply_color_to_outliers(output_excel, df, score_columns, threshold)
    print(f"Updated Excel file saved to {output_excel}")

# Example usage:
# save_outliers_to_excel('input.xlsx', ['col1', 'col2', 'col3', 'col4'], 0.7)


In [81]:
save_outliers_to_excel('consistency_9_11.xlsx', ['Rephrased query.1', 'Rephrased query.2', 'Rephrased query.3', 'Rephrased query.4'], 0.9, 'consistency_rephrased_query.xlsx')

[('Rephrased query.1', 'Rephrased query.2'), ('Rephrased query.1', 'Rephrased query.3'), ('Rephrased query.1', 'Rephrased query.4'), ('Rephrased query.2', 'Rephrased query.3'), ('Rephrased query.2', 'Rephrased query.4'), ('Rephrased query.3', 'Rephrased query.4')]


/var/folders/45/606m4h496zx145vhz0ndd4f40000gp/T/ipykernel_3847/1646262522.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Outlier Response'] = ', '.join(list(set([c for c in outlier_column_info if outlier_column_info.count(c) >= 2])))


Updated Excel file saved to consistency_rephrased_query.xlsx


In [78]:
save_outliers_to_excel('consistency.xlsx', ['Response_1', 'Response_2', 'Response_3', 'Response_4'], 0.7, 'consistency_9_11.xlsx')

[('Response_1', 'Response_2'), ('Response_1', 'Response_3'), ('Response_1', 'Response_4'), ('Response_2', 'Response_3'), ('Response_2', 'Response_4'), ('Response_3', 'Response_4')]
Updated Excel file saved to consistency_9_11.xlsx


In [50]:
save_outliers_to_excel('nvinfo_stg_sep10.xlsx', ['Response', 'Correct Answer'], None, 'nvinfo_stg_sep10.xlsx')

1
2
[('Response', 'Correct Answer')]
4
Updated Excel file saved to nvinfo_stg_sep10.xlsx


In [53]:
save_outliers_to_excel('nvinfo_stg_sep10.xlsx', ['Response', 'Response Reference'], 0.7, 'nvinfo_stg_sep10.xlsx')

[('Response', 'Response Reference')]
Updated Excel file saved to nvinfo_stg_sep10.xlsx


['asa']

In [69]:
file_names = ["after_1.xlsx", "after_2.xlsx", "after_3.xlsx","after_4.xlsx"]

evaluators =  [
        {
          "columns": [
            "Chat History Query",
            "Rephrased query",
            "Ranked IR Chunks",
            "Response"
          ],
          "scorers": [
            "bert"
          ],
          "threshold": 0.7
        },
        {
          "columns": [
            "Pre-facto experts"
          ],
          "scorers": [
            "em"
          ]
        }
      ]
columns = evaluators[0].get("columns") + evaluators[1].get("columns")



In [74]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_and_sort_excel(file_name: str, sort_column: str = None) -> pd.DataFrame:
    """
    Loads and sorts the Excel file by the specified sort column (default is the first column).
    
    :param file_name: The path to the Excel file.
    :param sort_column: The column name to sort by. If None, sorts by the first column.
    :return: Sorted DataFrame.
    """
    df = pd.read_excel(file_name)
    
    # Sort by the specified column, or the first column if not specified
    if sort_column is None:
        sort_column = df.columns[0]  # Sort by the first column if not specified
    
    df_sorted = df.sort_values(by=sort_column)  # Sort by the specified column
    return df_sorted

def extract_columns(df: pd.DataFrame, columns: list, file_index: int = None, column_suffix_separator: str = "_") -> pd.DataFrame:
    """
    Extracts columns from the DataFrame and labels them based on the file index.
    
    :param df: Input DataFrame.
    :param columns: List of column names to extract.
    :param file_index: Index to append to the column name.
    :param column_suffix_separator: String to use as the separator between column names and file index.
    :return: Extracted DataFrame with renamed columns.
    """
    extracted_columns = {}
    for col in columns:
        if col in df.columns:
            col_label = f"{col}{column_suffix_separator}{file_index}" if file_index is not None else col
            extracted_columns[col_label] = df[col]
        else:
            print(f"Error: Column '{col}' not found in file {file_index}")
    return pd.DataFrame(extracted_columns)

COLUMN_COMPARISON_SEPARATOR = ":"

def find_max_count(strings):
    """
    Finds the most common strings in a list.
    """
    string_counts = Counter(strings)
    max_count = max(string_counts.values())
    most_common_strings = [string for string, count in string_counts.items() if count >= max_count]
    return most_common_strings, max_count


def calculate_tfidf_similarity(list1, list2):
    """
    Calculate similarity between two lists using TF-IDF and Cosine Similarity.
    The order of the items in the list matters.
    """
    # Join list items into a single string, preserving order
    list1_str = ' '.join(list1)
    list2_str = ' '.join(list2)
    
    # Use TF-IDF vectorizer to convert text to vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([list1_str, list2_str])
    
    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    
    return cosine_sim[0][0]


def add_bert_f1_to_dataframe(df: pd.DataFrame, col1: str, col2: str, score_col: str) -> pd.DataFrame:
    """
    Adds a new column with BERT F1 scores to the DataFrame.
    
    :param df: DataFrame containing the two columns to compare.
    :param col1: Name of the first column (ground truth).
    :param col2: Name of the second column (predicted).
    :param score_col: Name of the column to store the BERT F1 score.
    :return: DataFrame with the BERT F1 score column added.
    """
    # Extract the ground truth and predicted columns from the DataFrame
    gt_list = [str(c) for c in df[col1].tolist()]
    pt_list = [str(c) for c in df[col2].tolist()]
    
    # Calculate BERT F1 scores
    _, _, f1 = score(gt_list, pt_list, lang="en", model_type="bert-base-uncased", batch_size=64)
    
    # Convert the torch.Tensor to a list and add it as a new column
    df[score_col] = f1.tolist()
    
    return df


def add_similarity_score(df: pd.DataFrame, col1: str, col2: str, score_col: str) -> pd.DataFrame:
    """
    Adds a new column with custom similarity scores to the DataFrame.
    Handles lists, dictionaries, and regular strings.
    
    :param df: DataFrame containing the two columns to compare.
    :param col1: Name of the first column (ground truth).
    :param col2: Name of the second column (predicted).
    :param score_col: Name of the column to store the similarity score.
    :return: DataFrame with the similarity score column added.
    """
    similarity_scores = []

    sim_score = None
    for idx, row in df.iterrows():
        value1 = row[col1]
        value2 = row[col2]

        # Check if both values are lists
        if isinstance(value1, list):
            sim_score = calculate_tfidf_similarity(value1, value2)
        
        # Check if both values are dictionaries
        elif isinstance(value1, dict):
            sim_score = 0
            if set(value1.keys()) == set(value2.keys()):
                
                dict1 = value1
                dict2 = value2
            
                sim_scores = []
                for key in dict1.keys():
                    v1 = dict1[key] if isinstance(dict1[key], list) else [str(value1)]
                    v2 = dict2[key] if isinstance(dict2[key], list) else [str(value1)]
        
                    _, _, key_f1 = score(v1, v2, lang="en", model_type="bert-base-uncased", batch_size=64)
                    sim_scores.append(key_f1[0].item())
        
                sim_score = sum(sim_scores) / len(sim_scores)
        
        similarity_scores.append(sim_score)
        
    if len(similarity_scores) == 0:
  
        _, _, f1 = score([str(c) for c in df[col1].tolist()], [str(c) for c in df[col1].tolist()], lang="en", model_type="bert-base-uncased", batch_size=64)
        similarity_scores = f1.tolist()
        
    # Add the similarity score to the DataFrame
    df[score_col] = similarity_scores

    return df


def align_comparison_columns(dfs: list, comparison_columns: dict, column_suffix_separator: str = "_") -> pd.DataFrame:
    """
    Aligns comparison columns from all files side by side, ensuring they are grouped for easy comparison.
    Adds pairwise comparisons and outlier columns using BERT F1 scores.
    
    :param dfs: List of DataFrames (from each file).
    :param comparison_columns: Dictionary where keys are column names and values are thresholds.
    :param column_suffix_separator: Separator used for naming columns.
    :return: A DataFrame with aligned comparison columns and outlier detection.
    """
    aligned_columns = pd.DataFrame()
    thresholds = {key: comparison_columns.get(key, 0.7) for key in comparison_columns}
    
    # Collect aligned comparison columns
    aligned_dfs = []
    for col in comparison_columns:
        aligned_col_group = []
        col_labels = []  # To keep track of column labels
        for i, df in enumerate(dfs, start=1):
            col_label = f"{col}{column_suffix_separator}{i}"
            if col_label in df.columns:
                aligned_col_group.append(df[col_label])
                col_labels.append(col_label)
            else:
                print(f"Error: Column '{col_label}' not found in file {i}")
        
        # Concatenate the aligned columns for this specific comparison column
        aligned_dfs.append(pd.concat(aligned_col_group, axis=1))
        
        # Step 1: Pairwise comparison for the aligned columns using BERT F1 scores
        if len(col_labels) > 1:
            for i, col1 in enumerate(col_labels):
                for col2 in col_labels[i+1:]:
                    new_col_name = f"{col1}{COLUMN_COMPARISON_SEPARATOR}{col2}"
                    aligned_dfs[-1] = add_similarity_score(aligned_dfs[-1], col1, col2, new_col_name)
        
        # Step 2: Detect outliers based on BERT F1 score and thresholds
        for i, col_label in enumerate(col_labels):
            outlier_col_name = f"{col_label} Outlier"
            aligned_dfs[-1][outlier_col_name] = aligned_dfs[-1].apply(lambda row: row[col_label] < thresholds[col], axis=1)
    
    return pd.concat(aligned_dfs, axis=1)

def process_files(file_names: list, additional_columns: list, comparison_columns: dict, column_suffix_separator: str = "_", sort_column: str = None):
    """
    Processes multiple Excel files, extracts specified columns, and merges them into one DataFrame.
    
    :param file_names: List of Excel file paths.
    :param additional_columns: List of columns to extract from the first file.
    :param comparison_columns: Dictionary with column names as keys and threshold values as values.
    :param column_suffix_separator: Separator used for naming columns from different files.
    :param sort_column: The column to sort each DataFrame by.
    :return: A merged DataFrame with sorted and extracted data, including outlier detection.
    """
    # Step 1: Load and sort the first file (additional_columns)
    df_first = load_and_sort_excel(file_names[0], sort_column=sort_column)
    
    # Step 2: Extract additional_columns from the first file
    additional_data = extract_columns(df_first, additional_columns)
    
    # Step 3: Loop through the rest of the files and extract comparison_columns
    comparison_dataframes = []
    for i, file_name in enumerate(file_names[1:], start=1):
        df = load_and_sort_excel(file_name, sort_column=sort_column)
        comparison_data = extract_columns(df, list(comparison_columns.keys()), i, column_suffix_separator)
        comparison_dataframes.append(comparison_data)
    
    # Step 4: Align comparison columns side by side with pairwise comparisons and outliers
    df_aligned_comparison = align_comparison_columns(comparison_dataframes, comparison_columns, column_suffix_separator)
    
    # Step 5: Merge additional_data with aligned comparison columns
    df_final = pd.concat([additional_data, df_aligned_comparison], axis=1)
    
    return df_final

# Example usage:
file_names = ["after_1.xlsx", "after_2.xlsx", "after_3.xlsx", "after_4.xlsx"]
additional_columns = ['Query', 'Correct Answer']
comparison_columns = {
    'Chat History Query': 0.7, 
    'Rephrased query': 0.7, 
    'Ranked IR Chunks': 0.7, 
    'Response': 0.7, 
    'Pre-facto experts': 0.7
}
COLUMN_SUFFIX_SEPARATOR = "_"
SORT_COLUMN = "ID"  # Replace with the actual name of the column to sort by, or leave as None to use the first column.

# Run the process
df_final = process_files(file_names, additional_columns, comparison_columns, COLUMN_SUFFIX_SEPARATOR, sort_column=SORT_COLUMN)
print(df_final)

# Optionally, save the result to Excel
df_final.to_excel('merged_output_with_outliers.xlsx', index=False)

TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:

# Optionally, save the result to Excel
df_merged.to_excel('merged_output1.xlsx', index=False)